In [1]:
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd
import numpy as np
import re

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'
}

Size_of_Squad = []
Team_Name = []
Team_Link = []
Team_ID = []
Average_Age = []
Total_Market_Value = []
Foreigners = []
Season = []
season = 2000
while True:
  base_url = f"https://www.transfermarkt.co.uk/egyptian-premier-league/startseite/wettbewerb/EGY1/plus/?saison_id={season}"
  pageTree = requests.get(base_url, headers=headers)
  soup = BeautifulSoup(pageTree.content, 'html.parser')

  # Find the correct table
  table = soup.find('table', class_='items')

  # Check if table is found
  if table:
      rows = table.find_all('tr', class_=['odd', 'even'])

      # Iterate through each row in the table
      for row in rows:
          # Team name and link
          team_data = row.find('td', {'class': 'zentriert no-border-rechts'})
          if team_data:
              name = team_data.find('a').get('title')
              link = team_data.find('a').get('href')
              Team_Name.append(name)
              Team_Link.append("https://www.transfermarkt.co.uk" + link)
              team_id = re.findall(r'\d+', link)[0]
              Team_ID.append(team_id)

          # Extracting other stats: Squad Size, Average Age, Foreigners, Market Value
          stats = row.find_all('td', {'class': 'zentriert'})
          if stats and len(stats) >= 4:
              squad_size = stats[1].text.strip()
              age = stats[2].text.strip()
              foreign_players = stats[3].text.strip()
              Size_of_Squad.append(squad_size)
              Foreigners.append(foreign_players)
              Average_Age.append(age)

          # Extracting Average Age and Total Market Value
          market_data = row.find_all('td', {'class': 'rechts'})
          if market_data and len(market_data) >= 2:
              market_value = market_data[1].text.strip()
              Total_Market_Value.append(market_value)
              Season.append(season)

      print (season, "Scraped")
      season += 1

      if season > 2024:
        break

2000 Scraped
2001 Scraped
2002 Scraped
2003 Scraped
2004 Scraped
2005 Scraped
2006 Scraped
2007 Scraped
2008 Scraped
2009 Scraped
2010 Scraped
2011 Scraped
2012 Scraped
2013 Scraped
2014 Scraped
2015 Scraped
2016 Scraped
2017 Scraped
2018 Scraped
2019 Scraped
2020 Scraped
2021 Scraped
2022 Scraped
2023 Scraped
2024 Scraped


In [ ]:
import csv
from itertools import zip_longest
file_list = [Season, Team_Name,Team_ID, Size_of_Squad ,Average_Age, Foreigners, Total_Market_Value, Team_Link]
exported = zip_longest(*file_list)
with open ("Teams stats 2000 to 2024.csv", "w") as myfile:
  wr = csv.writer (myfile)
  wr.writerow(["Season", "Team Name" , "Team ID", "Size of Squad" , "Average Age" , "Foreigners", "Total Market Value", "URL"])
  wr.writerows(exported)

In [9]:
df = pd.read_csv("Teams stats 2000 to 2024.csv")

In [3]:
df.head()

,Season,Team Name,Team ID,Size of Squad,Average Age,Foreigners,Total Market Value,URL
0,2000,Ghazl El Mahalla,13446,25,25.8,0,-,https://www.transfermarkt.co.uk/ghazl-mahalla/...
1,2000,Ismaily SC,3595,28,24.6,3,-,https://www.transfermarkt.co.uk/ismaily-sc/sta...
2,2000,Mansoura SC,19466,22,23.9,0,-,https://www.transfermarkt.co.uk/mansoura-sc/st...
3,2000,Ittihad Alexandria SC,3963,28,25.0,2,-,https://www.transfermarkt.co.uk/ittihad-alexan...
4,2000,Qanah FC,24297,27,25.9,2,-,https://www.transfermarkt.co.uk/qanah-fc/start...


In [11]:
def clean_net_transfer(value):
    # Remove currency symbols
    value = value.replace('€', '').replace('+', '').replace('-', '')

    # Handle millions (M) and thousands (k), converting to millions
    if 'm' in value.lower():
        return float(value.lower().replace('m', ''))  # Already in millions, no need to multiply
    elif 'k' in value.lower():
        return float(value.lower().replace('k', '')) / 1000  # Convert thousands to millions
    elif value == '': # Check for empty strings after removing hyphens
        return 0.0 # Or any suitable value for missing data
    else:
        return float(value) / 1e6  # If the value is in units, convert it to millions

# Apply the function to clean and convert the 'net_transfer_record' column to millions
df['Total Market Value'] = df['Total Market Value'].apply(clean_net_transfer)

In [12]:
df.sample(10)

,Season,Team Name,Team ID,Size of Squad,Average Age,Foreigners,Total Market Value,URL
331,2019,Aswan SC,22507,43,25.7,5,2.45,https://www.transfermarkt.co.uk/aswan-sc/start...
230,2014,Wadi Degla FC,18234,38,25.2,3,5.10,https://www.transfermarkt.co.uk/wadi-degla-fc/...
19,2001,Zamalek SC,664,34,25.4,2,0.00,https://www.transfermarkt.co.uk/zamalek-sc/sta...
26,2001,Qanah FC,24297,25,25.1,2,0.00,https://www.transfermarkt.co.uk/qanah-fc/start...
67,2004,Ghazl El Mahalla,13446,24,25.8,0,0.00,https://www.transfermarkt.co.uk/ghazl-mahalla/...
271,2016,Petrojet,10957,37,25.6,3,4.83,https://www.transfermarkt.co.uk/petrojet/start...
279,2017,Al Ahly FC,7,45,26.2,5,27.75,https://www.transfermarkt.co.uk/el-ahly-kairo/...
213,2013,Ghazl El Mahalla,13446,36,27.8,2,3.45,https://www.transfermarkt.co.uk/ghazl-mahalla/...
66,2004,Ittihad Alexandria SC,3963,24,27.3,2,0.00,https://www.transfermarkt.co.uk/ittihad-alexan...
241,2014,El Nasr,23469,38,24.6,4,2.00,https://www.transfermarkt.co.uk/el-nasr/starts...


In [13]:
df.to_csv("Teams stats 2000 to 2024.csv", index=False)